连通组件（不在乎方向）求解，可以拆解为如下两步：   

1.将当前节点的id，发送给周围邻居；   
2.对所有聚合后的id集合以及当前id取最小，更新为当前节点属性；  

重复1～2直到收敛，案例流程如下：   
![avatar](./pic/连通组件.png)

In [1]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.graphx._
val sparkConf: SparkConf = new SparkConf().setMaster("local[2]")
val sc: SparkContext = new SparkContext(sparkConf)

sparkConf = org.apache.spark.SparkConf@62d99e07
sc = org.apache.spark.SparkContext@d87cb67


org.apache.spark.SparkContext@d87cb67

In [2]:
val initGraph=util.GraphGenerators.rmatGraph(sc,32,64).removeSelfEdges().mapVertices((vid,_)=>vid)

initGraph = org.apache.spark.graphx.impl.GraphImpl@56195ad


org.apache.spark.graphx.impl.GraphImpl@56195ad

In [3]:
val rstGraph=Pregel(initGraph,initialMsg=Long.MaxValue,activeDirection=EdgeDirection.Either)(
    vprog=(_,vd,aggMsg)=>math.min(vd,aggMsg),
    sendMsg=(edge)=>{
        if(edge.srcAttr<edge.dstAttr){
            Iterator((edge.dstId,edge.srcAttr)) 
        }else if(edge.dstAttr<edge.srcAttr){
            Iterator((edge.srcId,edge.dstAttr)) 
        }else{
            Iterator.empty
        }
    },
    mergeMsg=(a,b)=>math.min(a,b)
)

rstGraph = org.apache.spark.graphx.impl.GraphImpl@235df905


org.apache.spark.graphx.impl.GraphImpl@235df905

In [4]:
//结果
rstGraph.vertices.take(32)

Array((16,16), (22,16), (28,16), (30,16), (24,16), (18,16), (20,16), (26,16), (19,16), (21,16), (25,16), (29,16), (27,16), (23,16), (17,16), (31,16))

In [5]:
//官方api
initGraph.connectedComponents().vertices.take(32)

Array((16,16), (22,16), (28,16), (30,16), (24,16), (18,16), (20,16), (26,16), (19,16), (21,16), (25,16), (29,16), (27,16), (23,16), (17,16), (31,16))